In [1]:
# Vinicius da Silva - 206734
import gurobipy as gp
import numpy as np
import time

In [2]:
#inicializações básicas
model = gp.Model()
qtd_clientes_J = 200
qtd_fabricas_F = np.random.randint(qtd_clientes_J,2*qtd_clientes_J+1)
qtd_maquinas_L = np.random.randint(5,11) #usamos 11 pq a função randint é do tipo [menor,maior[
qtd_materiais_M = np.random.randint(5,11)
qtd_produtos_P = np.random.randint(5,11)

print(f"J={qtd_clientes_J}, F={qtd_fabricas_F}, L={qtd_maquinas_L}, M={qtd_materiais_M}, P={qtd_produtos_P}")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-29


In [3]:
# Parâmetros baseados nas inicializações

# demanda do cliente j do produto p
demanda_J_P = np.random.randint(10,21,size=(qtd_clientes_J,qtd_produtos_P))

# material m necessario para produzir produto p na maquina l
material_necessario_M_P_L = np.random.randint(1,6,size=(qtd_materiais_M,qtd_produtos_P,qtd_clientes_J))

# material m disponivel na fabrica f
material_disponivel_M_F = np.random.randint(800,1001,size=(qtd_materiais_M,qtd_fabricas_F))

#capacidade de produção c da máquina l na fábrica f
capacidade_disponivel_L_F = np.random.randint(80,101,size=(qtd_maquinas_L,qtd_fabricas_F))

#custo de produção do produto p usando a máquina l na fábrica f
custo_producao_P_L_F = np.random.randint(10,101,size=(qtd_produtos_P,qtd_maquinas_L,qtd_fabricas_F))

# custo de transporte do produto p da fábrica f até o cliente j
custo_transporte_P_F_J = np.random.randint(10,21,size=(qtd_produtos_P,qtd_fabricas_F,qtd_clientes_J))

In [4]:
# Variáveis de decisão

start_time = time.time()

# Com base no modelo visto em sala, vamos usar o custo produzido e custo transportado de forma separada.
# Assim também vamos utilizar a qtd produzida e transportada de forma separada, dado que a minimização da função 
# é baseada na soma das duas variáveis

qtd_produzida = model.addMVar((qtd_produtos_P,qtd_maquinas_L,qtd_fabricas_F),vtype=gp.GRB.CONTINUOUS,name="qtd_produzida")

qtd_transportada = model.addMVar((qtd_produtos_P,qtd_fabricas_F,qtd_clientes_J),vtype=gp.GRB.CONTINUOUS,name="qtd_transportada")

In [5]:
# Restrições

# Demanda do cliente: A demanda dos clientes j por cada produto p deve ser igual a soma das toneladas 
# de cada produto p transportadas de cada fábrica f

demanda = model.addConstrs((
    gp.quicksum(qtd_transportada[p][f][j] for f in list(range(qtd_fabricas_F))) == demanda_J_P[j][p]
    for p in list(range(qtd_produtos_P))
    for j in list(range(qtd_clientes_J))
),name='demanda')


In [6]:
# Restrições

# Quantidade de matéria prima: A quantidade de matéria prima m disponível em cada fábrica f deve ser maior ou igual
# à quantidade necessária para produzir todos os pedidos feitos (quantidade produzida * quantidade necessaria)

material_disponivel = model.addConstrs((
    gp.quicksum(
        gp.quicksum(material_necessario_M_P_L[m][p][l] * qtd_produzida[p][l][f] for p in list(range(qtd_produtos_P)))
        for l in list(range(qtd_maquinas_L))
    ) <= material_disponivel_M_F[m][f]
    for m in list(range(qtd_materiais_M))
    for f in list(range(qtd_fabricas_F))
),name='material_disponivel')

In [7]:
# Restrições

# Capacidade disponível de produção: A capacidade disponível de produção deve ser maior ou igual a quantidade total 
# de produção que deve ser realizada

capacidade_producao = model.addConstrs((
    gp.quicksum(qtd_produzida[p][l][f] for p in list(range(qtd_produtos_P))) <= capacidade_disponivel_L_F[l][f]
    for l in list(range(qtd_maquinas_L))
    for f in list(range(qtd_fabricas_F))
),name='capacidade_disponivel')

In [8]:
# Restrições

# Relação entre quantidade transportada e produzida: A quantidade transportada deve ser igual a quantidade produzida

relacao_produzida_transportada = model.addConstrs((
    gp.quicksum(qtd_produzida[p][l][f] for l in list(range(qtd_maquinas_L))) 
    == 
    gp.quicksum(qtd_transportada[p][f][j] for j in list(range(qtd_clientes_J)))
    for p in list(range(qtd_produtos_P))
    for f in list(range(qtd_fabricas_F))
),name='relacao_produzida_transportada')

In [9]:
# Função objetivo

# o objetivo é minimizar o custo total de produção de todos os produtos em todas as máquinas de todas as fábricas
# somado com o custo total de transporte de todos os produtos levados de todas as fábricas até todos os clientes

custo_total_producao = gp.quicksum(
    gp.quicksum(
        gp.quicksum(qtd_produzida[p][l][f] * custo_producao_P_L_F[p][l][f] for f in list(range(qtd_fabricas_F)))
        for l in list(range(qtd_maquinas_L))
    )
    for p in list(range(qtd_produtos_P))
)

custo_total_transporte = gp.quicksum(
    gp.quicksum(
        gp.quicksum(qtd_transportada[p][f][j] * custo_transporte_P_F_J[p][f][j] for j in list(range(qtd_clientes_J)))
        for f in list(range(qtd_fabricas_F))
    )
    for p in list(range(qtd_produtos_P))
)

model.setObjective(custo_total_producao + custo_total_transporte,sense=gp.GRB.MINIMIZE)

In [10]:
model.optimize()

end_time = time.time()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8775 rows, 524799 columns and 1207791 nonzeros
Model fingerprint: 0x54382a94
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.62s
Presolved: 8775 rows, 524799 columns, 1207791 nonzeros

Ordering time: 0.44s

Barrier statistics:
 AA' NZ     : 5.658e+05
 Factor NZ  : 3.540e+06 (roughly 240 MB of memory)
 Factor Ops : 3.528e+09 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   6.82791254e+08  0.00000000e+00  2.17e+04 0.00e+00  3.40e+03     1s
   1   7.15842391e+07 -2.81551014e+06  2.28e+03 4.26e-14  

In [ ]:
print(f"Quantidade de Variáveis  = {sum(qtd_produzida.shape) + sum(qtd_produzida.shape)}")
print(f"Quantidade de Restrições = {len(demanda) + len(material_disponivel) + len(capacidade_producao) + len(relacao_produzida_transportada)}")
print(f"Número de Iterações      = {int(model.IterCount)}")
print(f"Tempo de Execução        = {end_time - start_time} s")
print(f"Custo da Solução         = {model.objVal}")